In [37]:
import pandas as pd
from datetime import datetime
import threading
import time
from tqdm import tqdm
from scipy import stats
import numpy as np
import psycopg2

def backtest_code(code, benchmark = 2):
    rolling = 5
    benchmark_index = "QQQ"
    
    pivoted = res[[code, benchmark_index]]
    pivoted.ffill()
    ret = pivoted.pct_change(fill_method=None)
    try:
        ret['stock_price'] = pivoted[code]
        ret['index_price'] = pivoted[benchmark_index]
        ret['ret_diff'] = ret[code] - ret[benchmark_index]
        # type 1
        # ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
        # ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
        # ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
        
        # type 2
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

        # # check direction
        ret['lag10'] = ret['stock_price'].shift(5)
        ret['change_stock'] = (ret['stock_price'] - ret['lag10'])/ret['lag10']
        ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)
        
        # # check if the direction are the same, if not, tned to not being a mean reversion
        ret['lag10_twii'] = ret['index_price'].shift(5)
        ret['change_twii'] = (ret['index_price'] - ret['lag10_twii'])/ret['lag10_twii']
        ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
        ret['filter_2'] = ret.apply(lambda x: 1 if abs(x['change_stock']) < abs(x['change_twii']) else 1, axis=1)
            
        # check raise
        ret[f'{code}_lag60'] = ret['stock_price'].shift(-12)
        ret['target_60mins'] = (ret[f'{code}_lag60'] - ret['stock_price'])/ret['stock_price'] * ret['direction']
        ret = ret[(ret['stdize_ret_diff'] >= benchmark) & (ret['filter_1'] == 1) & (ret['filter_2'] == 1)][['target_60mins', 'stdize_ret_diff']].dropna()
        ret['code'] = code
        return ret
    except Exception as e:
        print(e)

DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
cursor = conn.cursor()
cursor.execute("SELECT da, code, cl from stock_price;")
conn.commit()
df_list = cursor.fetchall()
df = pd.DataFrame(df_list, columns=['da', 'code', 'cl'])
res = df.pivot(values='cl', columns='code', index='da')


code_list=list(set(res.columns))

dfs = []
for code in code_list:
    if code == 'QQQ': continue
    df = backtest_code(code)
    if len(df) == 0: continue
    dfs.append(df)
dfs

[]